# Load data

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

ccba = pd.read_csv("..\\..\\dataset\\training_dataset_first\\public_train_x_ccba_full_hashed.csv")
cdtx = pd.read_csv("..\\..\\dataset\\training_dataset_first\\public_train_x_cdtx0001_full_hashed.csv")
custinfo = pd.read_csv("..\\..\\dataset\\training_dataset_first\\public_train_x_custinfo_full_hashed.csv")
dp = pd.read_csv("..\\..\\dataset\\training_dataset_first\\public_train_x_dp_full_hashed.csv")
remit = pd.read_csv("..\\..\\dataset\\training_dataset_first\\public_train_x_remit1_full_hashed.csv")
y = pd.read_csv("..\\..\\dataset\\training_dataset_first\\train_y_answer.csv")
date = pd.read_csv("..\\..\\dataset\\training_dataset_first\\train_x_alert_date.csv")

In [12]:
print(y.shape , date.shape)
print(set(y.alert_key.unique()) - set(date.alert_key.unique()))

(23906, 2) (23906, 2)
set()


In [14]:
label = y.merge(date, how='inner', on='alert_key')
data = custinfo.merge(label, how='inner', on='alert_key')
data

,alert_key,cust_id,risk_rank,occupation_code,total_asset,AGE,sar_flag,date
0,171142,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bfeb016d17b16f37a8d8,3,12.0,241719.0,3,0,0
1,171152,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389718e6c99b10041c816,3,13.0,599497.0,6,0,0
2,171177,a6cdf6302aead77112013168c6d546d2df3bcb551956d22c96cf97652e581602,1,19.0,51160.0,4,0,0
3,171178,1a3efa69705f611c7ef2384a715c8142e2ee801cfec9df4eb21131c33fc247a7,3,9.0,3634343.0,6,0,0
4,171180,67f8cbb64dd3d447e992b1b299e0ceed3372188e47c88e213b16abaedf6af700,1,17.0,4076287.0,4,0,0
...,...,...,...,...,...,...,...,...
23901,352123,b600c0720096c241f5aec16b096c6a353492eee8a5855aa7f79d81390811d914,1,17.0,12207.0,2,0,364
23902,352124,06c5ea5ccda4dfd8839c0dec8646fb3071d2c02883ef8a9f715aaa74f38d2419,1,17.0,259985.0,4,0,364
23903,352125,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c24bb3cbd245eaf8aa16,3,19.0,928963.0,3,0,364
23904,352128,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42dd8089cffc788b4a9e,3,19.0,21647.0,4,0,364


---

In [15]:
cdtx_1 = cdtx[(cdtx['country'] == 130) & (cdtx['cur_type'] == 47)].copy()

In [16]:
cdtx_2 = cdtx[(cdtx['country'] == 130) & (cdtx['cur_type'] != 47)].copy()

In [17]:
cdtx_3 = cdtx[(cdtx['country'] != 130) & (cdtx['cur_type'] == 47)].copy()

In [18]:
cdtx_4 = cdtx[(cdtx['country'] != 130) & (cdtx['cur_type'] != 47)].copy()

In [126]:
a = cdtx[(cdtx['cust_id'] == '0015cc01f553512007705edf855baa16f1b57a16fb107e0882c5cb0535d297a5') & (cdtx['date'] <= 394) & (cdtx['country'] == 130) & (cdtx['cur_type'] == 47)]
b = a.amt.count()
b

84

---

In [127]:
def transform_tx(row):
    ## tw_ntd data
    temp_tw_ntd = cdtx[(cdtx['date'] <= row['date']) & (cdtx['cust_id'] == row['cust_id']) & (cdtx['country'] == 130) & (cdtx['cur_type'] == 47)]
    row['avg_amt_tw_ntd'] = temp_tw_ntd.amt.mean()
    row['sd_amt_tw_ntd'] = temp_tw_ntd.amt.std()
    row['total_amt_tw_ntd'] = temp_tw_ntd.amt.sum()

    freq_tw_ntd = temp_tw_ntd.groupby(['date'], as_index=False).amt.count()
    row['avg_freq_tw_ntd'] = freq_tw_ntd.amt.mean()
    row['sd_freq_tw_ntd'] = freq_tw_ntd.amt.std()
    row['total_freq_tw_ntd'] = freq_tw_ntd.amt.sum()

    ## tw_nonntd data
    temp_tw_nonntd = cdtx[(cdtx['date'] <= row['date']) & (cdtx['cust_id'] == row['cust_id']) & (cdtx['country'] == 130) & (cdtx['cur_type'] != 47)]
    row['avg_amt_tw_nonntd'] = temp_tw_nonntd.amt.mean()
    row['sd_amt_tw_nonntd'] = temp_tw_nonntd.amt.std()
    row['total_amt_tw_nonntd'] = temp_tw_nonntd.amt.sum()

    freq_tw_nonntd = temp_tw_nonntd.groupby(['date'], as_index=False).amt.count()
    row['avg_freq_tw_nonntd'] = freq_tw_nonntd.amt.mean()
    row['sd_freq_tw_nonntd'] = freq_tw_nonntd.amt.std()
    row['total_freq_tw_nonntd'] = freq_tw_nonntd.amt.sum()

    ## nontw_ntd data
    temp_nontw_ntd = cdtx[(cdtx['date'] <= row['date']) & (cdtx['cust_id'] == row['cust_id']) & (cdtx['country'] != 130) & (cdtx['cur_type'] == 47)]
    row['avg_amt_nontw_ntd'] = temp_nontw_ntd.amt.mean()
    row['sd_amt_nontw_ntd'] = temp_nontw_ntd.amt.std()
    row['total_amt_nontw_ntd'] = temp_nontw_ntd.amt.sum()

    freq_nontw_ntd = temp_nontw_ntd.groupby(['date'], as_index=False).amt.count()
    row['avg_freq_nontw_ntd'] = freq_nontw_ntd.amt.mean()
    row['sd_freq_nontw_ntd'] = freq_nontw_ntd.amt.std()
    row['total_freq_nontw_ntd'] = freq_nontw_ntd.amt.sum()

    ## nontw_nonntd data
    temp_nontw_nonntd = cdtx[(cdtx['date'] <= row['date']) & (cdtx['cust_id'] == row['cust_id']) & (cdtx['country'] != 130) & (cdtx['cur_type'] != 47)]
    row['avg_amt_nontw_nonntd'] = temp_nontw_nonntd.amt.mean()
    row['sd_amt_nontw_nonntd'] = temp_nontw_nonntd.amt.std()
    row['total_amt_nontw_nonntd'] = temp_nontw_nonntd.amt.sum()

    freq_nontw_nonntd = temp_nontw_nonntd.groupby(['date'], as_index=False).amt.count()
    row['avg_freq_nontw_nonntd'] = freq_nontw_nonntd.amt.mean()
    row['sd_freq_nontw_nonntd'] = freq_nontw_nonntd.amt.std()
    row['total_freq_nontw_nonntd'] = freq_nontw_nonntd.amt.sum()

    return row

In [130]:
from tqdm import tqdm

tqdm.pandas(desc='Progress Bar')
custinfo_cdtx_agg = data.copy()
custinfo_cdtx_agg = custinfo_cdtx_agg.progress_apply(transform_tx,axis=1)

Progress Bar: 100%|██████████| 23906/23906 [2:08:17<00:00,  3.11it/s]  


In [132]:
custinfo_cdtx_agg.to_pickle('custinfo_cdtx_agg.pkl')